In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
import joblib
import pandas as pd
import tensorflow as tf

In [2]:
# Create sqlalchemy
from sqlalchemy import create_engine
engine = create_engine(f"sqlite:///spotify.sqlite")

In [3]:
# Connect to database
with engine.connect() as conn:
    songs_df = pd.read_sql("SELECT * FROM songs", conn)

In [4]:
songs_df.head()

,track_id,explicit,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,hit,genre
0,4pR4oQJULf7FDB54TleMyH,0,0.443,0.778,-7.564,0.2660,0.241,0.000000,0.2150,0.628,128.250,1.0,Hip_Hop_R&B
1,4Ds4bq6aqOSAp1T7DikOi4,0,0.580,0.800,-7.528,0.3450,0.281,0.000000,0.0708,0.483,81.303,0.0,Hip_Hop_R&B
2,7nYbKsvhM88mHCmFsAIQVp,0,0.805,0.746,-5.211,0.1850,0.238,0.000000,0.2150,0.717,127.922,0.0,Hip_Hop_R&B
3,7gVNP7rI9UBZndge0ulKfL,0,0.582,0.660,-4.988,0.0544,0.399,0.000000,0.1270,0.484,83.789,0.0,Hip_Hop_R&B
4,1hLvWelTny8vttEEZIXVjw,0,0.776,0.314,-9.513,0.0340,0.881,0.000723,0.1390,0.476,124.055,0.0,Hip_Hop_R&B


In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(songs_df.genre.values.reshape(-1,1)))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(['genre'])
encode_df.head()

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,genre_Cinema,genre_Classical,genre_Country,genre_Dance,genre_Foreign,genre_Hip_Hop_R&B,genre_Indie,genre_Jazz_Blues,genre_Kids,genre_Latin,genre_Pop,genre_Reggae,genre_Rock
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Merge the dataframes and drop genre
songs_df = songs_df.merge(encode_df, left_index=True, right_index=True).drop("genre",1)

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [7]:
songs_df.head()

,track_id,explicit,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,...,genre_Dance,genre_Foreign,genre_Hip_Hop_R&B,genre_Indie,genre_Jazz_Blues,genre_Kids,genre_Latin,genre_Pop,genre_Reggae,genre_Rock
0,4pR4oQJULf7FDB54TleMyH,0,0.443,0.778,-7.564,0.2660,0.241,0.000000,0.2150,0.628,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4Ds4bq6aqOSAp1T7DikOi4,0,0.580,0.800,-7.528,0.3450,0.281,0.000000,0.0708,0.483,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7nYbKsvhM88mHCmFsAIQVp,0,0.805,0.746,-5.211,0.1850,0.238,0.000000,0.2150,0.717,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7gVNP7rI9UBZndge0ulKfL,0,0.582,0.660,-4.988,0.0544,0.399,0.000000,0.1270,0.484,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1hLvWelTny8vttEEZIXVjw,0,0.776,0.314,-9.513,0.0340,0.881,0.000723,0.1390,0.476,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Scale tempo and loudness

scaled_features = songs_df.copy()
col_names = ['loudness', 'tempo']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

scaled_features[col_names] = features
scaled_features.head()

# Export scaler model to joblib
joblib.dump(scaler, './scaler_model.joblib')

['./scaler_model.joblib']

In [9]:
# Split our preprocessed data into our features and target arrays
y = songs_df['hit']
X = songs_df.drop(['hit', 'track_id'],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


# Logistic Regression

In [10]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

In [11]:
# Train the model
log_classifier.fit(X_train,y_train)

C:\Users\admin\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(max_iter=200)

In [12]:
# Make predictions
y_pred = log_classifier.predict(X_test)
results = pd.DataFrame({'Prediction': y_pred, 'Actual': y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,1.0
8,0.0,0.0
9,0.0,0.0


In [13]:
# Get accuracy score
print(accuracy_score(y_test,y_pred))

0.7410638297872341


In [14]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[17170,   223],
       [ 5862,   245]], dtype=int64)

In [15]:
# Print the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.75      0.99      0.85     17393
         1.0       0.52      0.04      0.07      6107

    accuracy                           0.74     23500
   macro avg       0.63      0.51      0.46     23500
weighted avg       0.69      0.74      0.65     23500



# Random Forest

In [16]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

In [17]:
# Train the model
rf_model = rf_model.fit(X_train, y_train)

In [18]:
# Make predictions
y_pred = rf_model.predict(X_test)
results = pd.DataFrame({'Prediction': y_pred, 'Actual': y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,1.0,1.0
8,0.0,0.0
9,0.0,0.0


In [19]:
# Get accuracy score
print(accuracy_score(y_test,y_pred))

0.8181702127659575


In [20]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[16724,   669],
       [ 3604,  2503]], dtype=int64)

In [21]:
# Print the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.82      0.96      0.89     17393
         1.0       0.79      0.41      0.54      6107

    accuracy                           0.82     23500
   macro avg       0.81      0.69      0.71     23500
weighted avg       0.81      0.82      0.80     23500



In [22]:
# Display sorted list of features and their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.10764845585869111, 'acousticness'),
 (0.1068401642478326, 'loudness'),
 (0.10580899667576897, 'danceability'),
 (0.10563880500973927, 'valence'),
 (0.10556492175569908, 'tempo'),
 (0.10442984020497302, 'speechiness'),
 (0.10228676621560005, 'energy'),
 (0.09967636320720807, 'liveness'),
 (0.08083872463254509, 'instrumentalness'),
 (0.014136628853099319, 'genre_Pop'),
 (0.008275377660525194, 'genre_Latin'),
 (0.0074984392733323845, 'explicit'),
 (0.006868088864383559, 'genre_Rock'),
 (0.006441529148060546, 'genre_Dance'),
 (0.0062558789710192575, 'genre_Foreign'),
 (0.005040037735601135, 'genre_Indie'),
 (0.004734999054576648, 'genre_Cinema'),
 (0.004285399325189628, 'genre_Classical'),
 (0.004174923282366262, 'genre_Hip_Hop_R&B'),
 (0.004098881459420741, 'genre_Kids'),
 (0.0034271307018306106, 'genre_Reggae'),
 (0.003317168518039612, 'genre_Country'),
 (0.0027124793444978776, 'genre_Jazz_Blues')]

In [23]:
# Export random forest model to joblib
joblib.dump(rf_model, './random_forest.joblib', compress=3)

['./random_forest.joblib']